In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
import seaborn as sns
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.impute import SimpleImputer, KNNImputer

In [ ]:
ethnic = pd.read_csv('path\\f.21000.tab', sep="\t")
ethnic = ethnic[['f.eid', 'f.21000.0.0']]

In [ ]:
perdicardial = pd.read_csv('path\\UKB_pericardial_fat_predictions.csv')
vat = pd.read_csv('path\\f.22407.tab', sep="\t")
asat = pd.read_csv('path\\f.22408.tab', sep="\t")

In [ ]:
bmi = pd.read_csv('path\\obesity_bmi_height_weight_cmr_phenotypes_45K.csv')
waist = pd.read_csv('path\\Obesity\\f.48.tab', sep="\t")
hip = pd.read_csv('path\\f.49.tab', sep="\t")

In [ ]:
bmi_ = bmi[['f.eid','BMI']]
waist_ = waist[['f.eid', 'f.48.2.0']]
hip_ = hip[['f.eid', 'f.49.2.0']]

bmi_waist = bmi_.merge(waist_)
bmi_waist_hip = bmi_waist.merge(hip_)

bmi_waist_hip['waist_hip_ratio'] = bmi_waist_hip['f.48.2.0']/bmi_waist_hip['f.49.2.0']

In [ ]:
p = perdicardial.merge(vat, on='f.eid')
pp = p.merge(asat, on='f.eid')
ppp = pp.merge(bmi_waist_hip, on='f.eid')

In [ ]:
ppp['pericardial'] = ppp['meanArea (cm2)']
ppp['VAT'] = ppp['f.22407.2.0']
ppp['ASAT'] = ppp['f.22408.2.0']
ppp['waist'] = ppp['f.48.2.0']
ppp['hip'] = ppp['f.49.2.0']

In [ ]:
ppp = ppp.drop(columns=['Unnamed: 0', 'date', 'predicted DSC', 'stdArea (cm2)', 'f.22407.3.0', 'f.22408.3.0', 'f.22407.2.0', 'f.22408.2.0', 'f.48.2.0', 'f.49.2.0', 'meanArea (cm2)'])
ppp = ppp.sort_values(by='f.eid').reset_index(drop=True)

In [ ]:
ids_after_cvd = pd.read_csv('path/ids_cvd_after_image_taken.csv')
covariates = pd.read_csv('path/ukb_covariates_34K.csv')
delta_female = pd.read_csv('path/delta_heart_female.csv')
delta_male = pd.read_csv('path/delta_heart_male.csv')
date_assit = pd.read_csv('path\\f.53.tab', sep="\t")

In [ ]:
cmr_phenotypes_34496 = pd.read_csv('path\\cmr_phenotypes_34496.csv')

In [ ]:
delta = pd.concat([delta_female, delta_male])
delta_cova = delta.merge(covariates, on='f.eid', how='left')
delta_cova = delta_cova.merge(ppp, on='f.eid')

In [ ]:
delta_cova['cvd'] = delta_cova['f.eid'].isin(ids_after_cvd['f.eid']).astype(int)

In [ ]:
delta_dates_cvd = delta_cova.merge(ids_after_cvd, on='f.eid', how='left')

In [ ]:
delta_dates_cvd = delta_dates_cvd.merge(date_assit[['f.eid', 'f.53.2.0']], on='f.eid')

In [ ]:
date_columns = ['event_date', 'f.42008.0.0', 'f.42010.0.0', 'f.42006.0.0', 'f.42000.0.0']
delta_dates_cvd[date_columns] = delta_dates_cvd[date_columns].apply(pd.to_datetime, errors='coerce')
delta_dates_cvd['date_event'] = delta_dates_cvd[date_columns].min(axis=1)
delta_dates_cvd['f.53.2.0'] = delta_dates_cvd['f.53.2.0_y'].astype(str)
delta_dates_cvd['f.53.2.0'] = delta_dates_cvd['f.53.2.0_y'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0]
delta_dates_cvd['date_event'] = delta_dates_cvd['date_event'].replace(np.nan,'31-10-2022')

In [ ]:
delta_dates_cvd['date_event'] = pd.to_datetime(delta_dates_cvd['date_event'], errors='coerce')
delta_dates_cvd['f.53.2.0'] = pd.to_datetime(delta_dates_cvd['f.53.2.0'], errors='coerce')

In [ ]:
delta_dates_cvd["time_to_event"] = (delta_dates_cvd["date_event"] - delta_dates_cvd["f.53.2.0"]).dt.days

In [ ]:
delta_dates_cvd['age'] = delta_dates_cvd['age_at_recruitment_visit2_x']

In [ ]:
delta_dates_cvd = delta_dates_cvd.sort_values(by='f.eid').reset_index(drop=True)

In [ ]:
data = delta_dates_cvd[['f.eid', 'delta', 'cvd', 'time_to_event', 'age', 'sex', 'smoking_status', 'ethnicity', 'physical_moderate']]

In [ ]:
data['cvd'].value_counts()

In [ ]:
data = data.merge(ppp, on='f.eid')

In [ ]:
data['smoking_status'].value_counts()

In [ ]:
data['ethnicity'].value_counts()

In [ ]:
data['ethnicity'] = data['ethnicity'].replace('unknown', np.nan)

In [ ]:
data['ethnicity'].value_counts()

In [ ]:
mask_nan = data['ethnicity'].isna()
encoder = LabelEncoder()
data.loc[~mask_nan, 'ethnicity'] = encoder.fit_transform(data.loc[~mask_nan, 'ethnicity'])
data.loc[mask_nan, 'ethnicity'] = np.nan

In [ ]:
encoder = LabelEncoder()
#data['ethnicity'] = encoder.fit_transform(data['ethnicity'])
data['smoking_status'] = encoder.fit_transform(data['smoking_status'])

In [ ]:
data['ethnicity'].value_counts()

In [ ]:
data['smoking_status'].value_counts()

In [ ]:
data['smoking_status'] = data['smoking_status'].replace(4, np.nan)

In [ ]:
#data['ethnicity'] = data['ethnicity'].replace(5, np.nan)

In [ ]:
data['ethnicity'].value_counts()

In [ ]:
data['smoking_status'].value_counts()

In [ ]:
sex = pd.read_csv('C:\\Users\\Cynthia Maldonado\\OneDrive - Queen Mary, University of London\\Biological Aging\\data_ukbb\\CVD\\sex_34496.csv')
data = data.drop(columns='sex')
data = sex.merge(data, on='f.eid', how='left')

In [ ]:
data['sex'].value_counts()

In [ ]:
data.isna().sum()

In [ ]:
imp_cat = SimpleImputer(strategy='most_frequent')
imp_num = KNNImputer(n_neighbors=10)

In [ ]:
cat_cols = ['ethnicity', 'smoking_status']
num_cols = [col for col in data.columns if col not in cat_cols]

data_cat_imputed = pd.DataFrame(imp_cat.fit_transform(data[cat_cols]), columns=cat_cols)

data_num_imputed = pd.DataFrame(imp_num.fit_transform(data[num_cols]), columns=num_cols)
data_imputed = pd.concat([data_num_imputed, data_cat_imputed], axis=1)
survival_data= data_imputed[data.columns]

In [ ]:
survival_data['ethnicity'].value_counts()

In [ ]:
survival_data['smoking_status'].value_counts()

In [ ]:
survival_data['cvd'].value_counts()

In [ ]:
scaler = StandardScaler()

In [ ]:
survival_data.isna().sum()

In [ ]:
cols_to_scale = ['VAT', 'ASAT', 'pericardial', 'BMI', 'waist', 'hip', 'delta', 'age', 'sex', 'smoking_status', 'ethnicity', 'physical_moderate'] 
survival_data[cols_to_scale] = scaler.fit_transform(survival_data[cols_to_scale])
columns_obesity = ['VAT', 'ASAT', 'pericardial', 'BMI', 'waist', 'hip', 'delta']
covariates = [ 'sex', 'physical_moderate', 'ethnicity', 'smoking_status' ] #'ethnicity', 'physical_moderate', 'sex']

In [ ]:
results_cox = []
for i in columns_obesity:
    model = CoxPHFitter()
    # Include both the obesity measure and covariates in the analysis
    survival_data_1 = survival_data[[i] + covariates + ['time_to_event', 'cvd']]
#    print(survival_data_1.columns)
    model.fit(survival_data_1, 'time_to_event', event_col='cvd')
    summary = model.summary
    beta = summary.loc[i, 'coef']
    HR = summary.loc[i, 'exp(coef)']
    CI_lower = summary.loc[i, 'exp(coef) lower 95%']
    CI_upper = summary.loc[i, 'exp(coef) upper 95%']
    p_value = summary.loc[i, 'p']
    
    # Append to list
    results_cox.append([i, beta, HR, CI_lower, CI_upper, p_value])
    
df_results_cox = pd.DataFrame(results_cox, columns=['Variable', 'Beta', 'HR', 'CI_lower', 'CI_upper', 'p_value'])

In [ ]:
df_results_cox

# Mediation

In [ ]:
survival_data['time_to_event_yrs'] = survival_data['time_to_event'] / 365.25

In [ ]:
def mediation_analysis(
    data, 
    exposures,  # List of exposure variables (e.g., ['BMI', 'VAT', 'waist_hip'])
    mediator,   # Name of mediator (e.g., 'cardiac_delta')
    outcome,    # Name of outcome (e.g., 'CVD_status')
    time_to_event,  # Name of time-to-event column (e.g., 'time_to_CVD_yrs')
    covariates, # List of covariates (e.g., ['age', 'sex', 'smoking'])
    n_bootstrap=1000,
    n_jobs=-1   # Use all CPU cores
):
    """
    Perform mediation analysis for multiple exposures using:
    - Linear regression for the mediator (M ~ X + covariates)
    - Cox PH model for the outcome (Y ~ X + M + covariates)
    - Bootstrapping to estimate indirect effects (a*b) and 95% CIs.
    
    Returns a DataFrame with direct, indirect, and total effects for each exposure.
    """
    results = []
    
    for exposure in exposures:
        # Step 1: Fit mediator model (M ~ X + covariates)
        mediator_formula = f"{mediator} ~ {exposure} + " + " + ".join(covariates)
        mediator_model = sm.OLS.from_formula(mediator_formula, data=data).fit()
        a = mediator_model.params[exposure]
        
        # Step 2: Fit outcome model (Y ~ X + M + covariates)
        cph = CoxPHFitter()
        outcome_data = data[[time_to_event, outcome, exposure, mediator] + covariates]
        cph.fit(outcome_data, duration_col=time_to_event, event_col=outcome)
        b_beta = cph.params_[mediator]           # log(HR) for mediator
        b_HR = np.exp(b_beta)
        c_prime = cph.params_[exposure]         # direct effect in log(HR)
        total_effect = a * b_beta + c_prime     # total effect in log(HR)
        indirect_effect = a * b_beta            # indirect effect in log(HR)
        
        # Step 3: Bootstrap indirect effect (a*b)
        def bootstrap_sample(data):
            sample = data.sample(n=len(data), replace=True)
            # Mediator model
            med_model = sm.OLS.from_formula(mediator_formula, data=sample).fit()
            a_boot = med_model.params[exposure]
            # Outcome model
            cph_boot = CoxPHFitter()
            cph_boot.fit(
                sample[[time_to_event, outcome, exposure, mediator] + covariates],
                duration_col=time_to_event,
                event_col=outcome
            )
            b_boot = cph_boot.params_[mediator]
            return a_boot * b_boot
        
        # Parallel bootstrapping
        bootstrap_effects = Parallel(n_jobs=n_jobs)(
            delayed(bootstrap_sample)(data) for _ in tqdm(range(n_bootstrap)))
        bootstrap_effects = np.array(bootstrap_effects)
        
        # Calculate CIs and proportion mediated
        lower_ci = np.percentile(bootstrap_effects, 2.5)
        upper_ci = np.percentile(bootstrap_effects, 97.5)
        proportion_mediated = indirect_effect / total_effect

         # Convert log(HR) to HR where applicable
        total_HR = np.exp(total_effect)
        direct_HR = np.exp(c_prime)
        indirect_HR = np.exp(indirect_effect)
        lower_HR = np.exp(lower_ci)
        upper_HR = np.exp(upper_ci)       
        
        # Store results
        results.append({
            'Exposure': exposure,
            'Total_beta': total_effect,
            'Total_HR': total_HR,
            'Direct_beta': c_prime,
            'Direct_HR': direct_HR,
            'Indirect_beta': indirect_effect,
            'Indirect_HR': indirect_HR,
            'a (X->M)': a,
            'b_beta (M->Y)': b_beta,
            'b_HR (M->Y)': b_HR,
            'Proportion_Mediated': proportion_mediated,
            'Indirect_95%CI_beta_lower': lower_ci,
            'Indirect_95%CI_beta_upper': upper_ci,
            'Indirect_95%CI_HR_lower': lower_HR,
            'Indirect_95%CI_HR_upper': upper_HR
        })

    return pd.DataFrame(results)

In [ ]:
if __name__ == "__main__":
    # Define variables
    exposures = ['BMI', 'VAT', 'pericardial', 'ASAT', 'waist', 'hip']  # Obesity measures
    mediator = 'delta'                # Mediator
    outcome = 'cvd'                    # Binary outcome
    time_to_event = 'time_to_event_yrs'         # Follow-up time in years
    covariates = ['sex', 'smoking_status', 'ethnicity', 'physical_moderate']    # Adjust for these
    
    # Run mediation analysis
    results = mediation_analysis(
        data=survival_data,
        exposures=exposures,
        mediator=mediator,
        outcome=outcome,
        time_to_event=time_to_event,
        covariates=covariates,
        n_bootstrap=1000  # Increase for final analysis
    )

In [ ]:
results